# Experiment: Which drugs are "similar", from prescription data?

In the USA the social program for prescription drugs is the "Medicare Part D Prescription Drug" program. Data regarding the prescribed drugs per provider and drug are available at https://data.cms.gov/provider-summary-by-type-of-service/medicare-part-d-prescribers/medicare-part-d-prescribers-by-provider-and-drug/

On this website you can select the columns you want. I downloaded only the code for the drug provider (`Prscrbr_NPI`), the type of provider (`Prscrbr_Type`), the generic name for the drug (`Gnrc_Name`) and the total number of claims made by this provider (`Tot_Clms`):

<center><img src='./image.png' width=50%></img></center>

As a result, I got a 197MB zip file. I made a folder named `data` and placed the file there. I also wrote a utility function to load this data and placed this code in a separate `.py` file (just so I don't pollute this notebook too much!). Let's load the data:

In [1]:
from pathlib import Path

import pandas as pd

from utils import DATA_DIR, load_data

In [2]:
df = load_data()

c:\Users\fjayr\Dropbox\Insper\Disciplinas\MachineLearning\Common\machine_learning\labs\01_SVD\labs\data\Medicare_Part_D_Prescribers_by_Provider_and_Drug_2022.csv not found, extracting zip file
Loading data from c:\Users\fjayr\Dropbox\Insper\Disciplinas\MachineLearning\Common\machine_learning\labs\01_SVD\labs\data\Medicare_Part_D_Prescribers_by_Provider_and_Drug_2022.csv


## Exploratory analysis

What does the data look like?

In [3]:
df.shape

(25869521, 4)

- About 25 million rows, 4 columns

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25869521 entries, 0 to 25869520
Data columns (total 4 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   Prscrbr_NPI   int64 
 1   Prscrbr_Type  object
 2   Gnrc_Name     object
 3   Tot_Clms      int64 
dtypes: int64(2), object(2)
memory usage: 789.5+ MB


- The columns are:

| Column Name  | Description                  | Dtype   | Type meaning |
|--------------|------------------------------|---------|--------------|
| Prscrbr_NPI  | Doctor's ID                  | int64   | an integer   |  
| Prscrbr_Type | Doctor type                  | object  | a string     | 
| Gnrc_Name    | Drug name                    | object  | a string     | 
| Tot_Clms     | Number of prescriptions made | int64   | an integer   | 

In [5]:
df.head(10)

,Prscrbr_NPI,Prscrbr_Type,Gnrc_Name,Tot_Clms
0,1861868291,NaN,Cefazolin Sodium,11
1,1861868291,NaN,Ceftriaxone Sodium,14
2,1912497371,NaN,Amoxicillin,13
3,1912497371,NaN,Bictegrav/Emtricit/Tenofov Ala,11
4,1912497371,NaN,Doxycycline Monohydrate,26
5,1013232149,Acupuncturist,Amlodipine Besylate,12
6,1013232149,Acupuncturist,Losartan Potassium,11
7,1013232149,Acupuncturist,Metoprolol Succinate,13
8,1013232149,Acupuncturist,Rosuvastatin Calcium,13
9,1063638625,Acupuncturist,Amlodipine Besylate,14


How many absent items (NaNs) are there in the dataset?

In [6]:
df.isna().sum()

Prscrbr_NPI     0
Prscrbr_Type    5
Gnrc_Name       0
Tot_Clms        0
dtype: int64

So there are only five absent items in the `Prscrbr_Type` column. Let's discard the corresponding rows:

In [7]:
df = df.dropna()

In [8]:
df.shape

(25869516, 4)

The `Prscrbr_NPI`, `Prscrbr_Type`, and `Gnrc_Name` contain categorical values. How many categories in each of them?

In [9]:
df[['Prscrbr_NPI', 'Prscrbr_Type', 'Gnrc_Name']].nunique()

Prscrbr_NPI     1057564
Prscrbr_Type        180
Gnrc_Name          1757
dtype: int64

If we were to create a matrix of `Prscrbr_NPI` by `Gnrc_Name`, with `Tot_Clms` for cell values, that matrix would be of size $1057564 \times 1757$! How many values a matrix of this size would store?

In [10]:
num_positions = 1057564 * 1757
print(f'{num_positions:,}')

1,858,139,948


Almost two billion positions! And most of them are zeros, because we only have $25,869,516$ non-zero values.

In [11]:
num_nonzero_positions = df.shape[0]
print(f'{num_nonzero_positions:,}')

25,869,516


The occupancy ratio of this matrix is very, very small:

In [12]:
occupancy_ratio = num_nonzero_positions / num_positions
print(f'{occupancy_ratio:.2%}')

1.39%


Only about $1.39\%$ of the matrix is non-zero. Such matrices are called *sparse matrices*. There are algorithmic techniques to handle sparse matrices in an efficient manner, both in terms of amount of computation to be performed and in memory usage. We will make use of it in a bit.

But we should also consider whether all of these categories in `Prscrbr_NPI` and `Gnrc_Name` are truly useful. Maybe categories with under-representation could be discarded, since we want to focus on the most present information only.

Let's start with the physician type:

In [13]:
count_Prscrbr_Type = df['Prscrbr_Type'].value_counts()

In [14]:
count_Prscrbr_Type.head(10)

Prscrbr_Type
Family Practice        6651765
Internal Medicine      5603062
Nurse Practitioner     5053595
Physician Assistant    1878231
Cardiology              710812
Psychiatry              547364
General Practice        407701
Neurology               386005
Endocrinology           271592
Ophthalmology           260765
Name: count, dtype: int64

There is a steep decline in representation from the most common type of doctor ("Family Practice") to the 10th most common ("Ophtalmology").

In [15]:
count_Prscrbr_Type.tail(10)

Prscrbr_Type
Clinical Medical Laboratory                              1
Assisted Living Facility                                 1
Doula                                                    1
Home Health Aide                                         1
Independent Diagnostic Testing Facility (IDTF)           1
Local Education Agency (LEA)                             1
Nursing Home Administrator                               1
Phlebology                                               1
Public Health or Welfare Agency                          1
Residential Treatment Facility, Physical Disabilities    1
Name: count, dtype: int64

The least represented types of doctor have only one entry each.

Let's decide on a cutoff value to filter the least common types. It will be quite arbitrary, and it is worthwhile to consider whether this removal of information will qualitatively impact the conclusions to be obtained from the analysis of this dataset - keep this in mind, ok?

In [16]:
import numpy as np

In [17]:
count_Prscrbr_Type.quantile(np.linspace(0, 1, 21))

0.00          1.00
0.05          1.00
0.10          2.00
0.15          6.00
0.20         11.00
0.25         19.00
0.30         28.00
0.35         46.30
0.40        124.20
0.45        193.30
0.50        465.50
0.55       1169.85
0.60       2337.00
0.65       4591.80
0.70       8970.40
0.75      18265.25
0.80      50342.20
0.85      99978.15
0.90     156544.30
0.95     261306.35
1.00    6651765.00
Name: count, dtype: float64

Making a list of quantiles from zero to $100\%$, in steps of $5\%$, we see an almost exponential growth in number of physicians per type. Let's cut aggressively then: it will not change the number of data points substantially.

In [18]:
q_Prscrbr_Type = 0.70
cut_value = count_Prscrbr_Type.quantile(q_Prscrbr_Type)

useful_Prscrbr_Type = count_Prscrbr_Type[count_Prscrbr_Type >= cut_value].index

In [19]:
useful_Prscrbr_Type

Index(['Family Practice', 'Internal Medicine', 'Nurse Practitioner',
       'Physician Assistant', 'Cardiology', 'Psychiatry', 'General Practice',
       'Neurology', 'Endocrinology', 'Ophthalmology', 'Nephrology',
       'Emergency Medicine', 'Dentist', 'Pulmonary Disease',
       'Gastroenterology', 'Hematology-Oncology', 'Dermatology', 'Urology',
       'Rheumatology', 'Hospitalist', 'Interventional Cardiology',
       'Geriatric Medicine',
       'Student in an Organized Health Care Education/Training Program',
       'Optometry', 'Obstetrics & Gynecology', 'Psychiatry & Neurology',
       'Orthopedic Surgery', 'Infectious Disease',
       'Physical Medicine and Rehabilitation', 'Otolaryngology', 'Podiatry',
       'General Surgery', 'Medical Oncology', 'Allergy/ Immunology',
       'Clinical Cardiac Electrophysiology', 'Pharmacist', 'Anesthesiology',
       'Pain Management', 'Pediatric Medicine',
       'Certified Clinical Nurse Specialist', 'Interventional Pain Management',
    

In [20]:
df = df[df['Prscrbr_Type'].isin(useful_Prscrbr_Type)]

In [21]:
df.shape

(25754507, 4)

We only reduced the number of data points from $25,869,516$ to $25,754,507$!

Now let's look into the physicians:

In [22]:
count_Prscrbr_NPI = df['Prscrbr_NPI'].value_counts()

In [23]:
count_Prscrbr_NPI.quantile(np.linspace(0, 1, 21))

0.00      1.0
0.05      1.0
0.10      1.0
0.15      1.0
0.20      2.0
0.25      2.0
0.30      3.0
0.35      4.0
0.40      5.0
0.45      6.0
0.50      8.0
0.55     10.0
0.60     13.0
0.65     17.0
0.70     22.0
0.75     29.0
0.80     39.0
0.85     53.0
0.90     75.0
0.95    111.0
1.00    590.0
Name: count, dtype: float64

Again, a very exponential-like growth of number of prescriptions per physician. Let's cut aggressively again, but remember: this may impact the quality of the subsequent analysis, so it may be interesting to revisit these arbitrary decisions later.

In [24]:
q_Prscrbr_NPI = 0.5
cut_value = count_Prscrbr_NPI.quantile(q_Prscrbr_NPI)

useful_Prscrbr_NPI = count_Prscrbr_NPI[count_Prscrbr_NPI >= cut_value].index

In [25]:
len(useful_Prscrbr_NPI)

537484

In [26]:
df = df[df['Prscrbr_NPI'].isin(useful_Prscrbr_NPI)]

In [27]:
df.shape

(24297207, 4)

We are keeping a large portion of the information still: $24,297,207$, from the previous number of $25,754,507$ rows.

Now let's investigate the drugs:

In [28]:
count_Gnrc_Name = df['Gnrc_Name'].value_counts()

In [29]:
count_Gnrc_Name.head(10)

Gnrc_Name
Levothyroxine Sodium    399226
Metformin Hcl           383512
Gabapentin              315303
Atorvastatin Calcium    307186
Albuterol Sulfate       301663
Amlodipine Besylate     299293
Lisinopril              286633
Losartan Potassium      270682
Omeprazole              260114
Furosemide              256761
Name: count, dtype: int64

In [30]:
count_Gnrc_Name.tail(10)

Gnrc_Name
Caplacizumab-Yhdp                 1
Lurbinectedin                     1
Belantamab Mafodotin-Blmf         1
Folic Acid/B Cplx/C/Selen/Zinc    1
Zinc/Copper/Manganese/Selenium    1
Evinacumab-Dgnb                   1
Pediatric Multivitamin No.17      1
Ciprofloxacin                     1
Ciprofloxacin Hcl/Fluocinolone    1
Methotrexate                      1
Name: count, dtype: int64

In [31]:
count_Gnrc_Name.quantile(np.linspace(0, 1, 21))

0.00         1.00
0.05         1.00
0.10         3.00
0.15         7.00
0.20        13.00
0.25        23.00
0.30        37.00
0.35        59.00
0.40       111.00
0.45       187.25
0.50       303.00
0.55       601.75
0.60       998.00
0.65      1627.50
0.70      2728.00
0.75      4894.50
0.80      7601.00
0.85     14959.00
0.90     33058.50
0.95     93739.25
1.00    399226.00
Name: count, dtype: float64

Again, an exponential growth in quantiles, let's cut:

In [32]:
q_Gnrc_Name = 0.5
cut_value = count_Gnrc_Name.quantile(q_Gnrc_Name)

useful_Gnrc_Name = count_Gnrc_Name[count_Gnrc_Name >= cut_value].index

In [33]:
useful_Gnrc_Name

Index(['Levothyroxine Sodium', 'Metformin Hcl', 'Gabapentin',
       'Atorvastatin Calcium', 'Albuterol Sulfate', 'Amlodipine Besylate',
       'Lisinopril', 'Losartan Potassium', 'Omeprazole', 'Furosemide',
       ...
       'Alectinib Hcl', 'Sulfacetamide Sodium', 'Fondaparinux Sodium',
       'Sub-Q Insulin Device, 30 Unit', 'Trametinib Dimethyl Sulfoxide',
       'Chlordiazepoxide/Clidinium Br', 'Erythromycin Base In Ethanol',
       'Lamivudine/Zidovudine', 'Emtricita/Rilpivirine/Tenof Df',
       'Dipyridamole'],
      dtype='object', name='Gnrc_Name', length=875)

In [34]:
df = df[df['Gnrc_Name'].isin(useful_Gnrc_Name)]

In [35]:
df.shape

(24246835, 4)

Finally, after all this filtering, we went from $25,869,521$ items to $24,246,835$.

In [36]:
print(f'We dropped {1.0 - 24246835/25869521:.2%} of the data')

We dropped 6.27% of the data


In [37]:
num_doctors = df['Prscrbr_NPI'].nunique()
print(f'There are {num_doctors:,} doctors left.')

There are 537,484 doctors left.


In [38]:
num_drugs = df['Gnrc_Name'].nunique()
print(f'There are {num_drugs:,} drugs left.')

There are 875 drugs left.


In [39]:
print( \
    f'There are {num_doctors:,} times {num_drugs:,} ' + \
    f'= {num_doctors * num_drugs:,} possible pairs (doctor, drug), ' + \
    'but the vast majority of them will have zero prescriptions.' \
)

There are 537,484 times 875 = 470,298,500 possible pairs (doctor, drug), but the vast majority of them will have zero prescriptions.


In [40]:
print( \
    f'There are {df.shape[0]:,} records left.' + \
    f' The occupancy ratio is {df.shape[0] / (num_doctors * num_drugs):.2%}.' \
)

There are 24,246,835 records left. The occupancy ratio is 5.16%.


Our matrix of prescriptions by doctors and drugs would still be sparse, but with a better occupancy ratio now.

Lets save the results of our exploratory analysis!

Let's create a sparse matrix from the dataframe. The trick is to convert the columns of interest to categorical data, so that Pandas will identify the unique doctors and drugs, and make an integer code for each one, from zero to the number of unique items minus one.

In [41]:
df = df.reset_index(drop=True)

In [42]:
df['Prscrbr_NPI'] = df['Prscrbr_NPI'].astype('category')
df['Gnrc_Name'] = df['Gnrc_Name'].astype('category')
df['Prscrbr_Type'] = df['Prscrbr_Type'].astype('category')
df['Tot_Clms'] = df['Tot_Clms'].astype(float)

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24246835 entries, 0 to 24246834
Data columns (total 4 columns):
 #   Column        Dtype   
---  ------        -----   
 0   Prscrbr_NPI   category
 1   Prscrbr_Type  category
 2   Gnrc_Name     category
 3   Tot_Clms      float64 
dtypes: category(3), float64(1)
memory usage: 367.1 MB


Let's see the categorical encoding in action. What are the drugs?

In [44]:
df['Gnrc_Name'].cat.categories

Index(['0.9 % Sodium Chloride', 'Abacavir Sulfate',
       'Abacavir Sulfate/Lamivudine', 'Abacavir/Dolutegravir/Lamivudi',
       'Abaloparatide', 'Abatacept', 'Abemaciclib', 'Abiraterone Acetate',
       'Acalabrutinib', 'Acamprosate Calcium',
       ...
       'Voriconazole', 'Vortioxetine Hydrobromide', 'Warfarin Sodium',
       'Zafirlukast', 'Zaleplon', 'Zanubrutinib', 'Ziprasidone Hcl',
       'Zolmitriptan', 'Zolpidem Tartrate', 'Zonisamide'],
      dtype='object', length=875)

So each drug is associated with an index number. In this case, "0.9 % Sodium Chloride" is drug number zero, "Abacavir Sulfate" is drug number one, and so on.

Let's see what is stored now in the dataframe:

In [45]:
df['Gnrc_Name'].head()

0          Amiodarone Hcl
1     Amlodipine Besylate
2    Atorvastatin Calcium
3     Bisoprolol Fumarate
4              Carvedilol
Name: Gnrc_Name, dtype: category
Categories (875, object): ['0.9 % Sodium Chloride', 'Abacavir Sulfate', 'Abacavir Sulfate/Lamivudine', 'Abacavir/Dolutegravir/Lamivudi', ..., 'Ziprasidone Hcl', 'Zolmitriptan', 'Zolpidem Tartrate', 'Zonisamide']

Looks like the same as before. But what about the equivalent codes?

In [46]:
df['Gnrc_Name'].cat.codes.head()

0     36
1     39
2     64
3     97
4    144
dtype: int16

Do the codes and names really correspond to each other? Sure they do! Look at this:

In [47]:
for code in df['Gnrc_Name'].cat.codes.head():
    print(f'code is {code}, name is {df['Gnrc_Name'].cat.categories[code]}')

code is 36, name is Amiodarone Hcl
code is 39, name is Amlodipine Besylate
code is 64, name is Atorvastatin Calcium
code is 97, name is Bisoprolol Fumarate
code is 144, name is Carvedilol


Now we can make a matrix of the data. We will use the doctor NPI as the row index and the drug name as the column index. The value of each cell will be the number of prescriptions.

Mathematically, this is a matrix like any other. Computationally, we will make a sparse matrix in the CSR - Compressed Sparse Row - format (https://en.wikipedia.org/wiki/Sparse_matrix)

In [48]:
row_index = df['Prscrbr_NPI'].cat.codes
num_rows = df['Prscrbr_NPI'].cat.categories.size

col_index = df['Gnrc_Name'].cat.codes
num_cols = df['Gnrc_Name'].cat.categories.size

values = df['Tot_Clms'].values

In [49]:
from scipy.sparse import csr_matrix

sparse_matrix = csr_matrix(
    (values, (row_index, col_index)),
    shape=(num_rows, num_cols),
)

In [50]:
sparse_matrix.shape

(537484, 875)

Now we are ready for the Singular Value Decomposition (SVD). What we want to do is to decompose our matrix $\bm{X} \in \mathbb{R}^{m \times n}$ of doctors per medication (where $m$ is the number of doctors and $n$ is the number of medications, $m \gg n$) into a product of three matrices: $\bm{U} \in \mathbb{R}^{m \times m}$, $S \in \mathbb{R}^{m \times n}$, and $V^{T} \in \mathbb{R}^{n \times n}$ such that:

- $\bm{X} = \bm{U} \bm{S} \bm{V}^{T}$
- Matrices $\bm{U}$ and $\bm{V}$ are unitary: 
    - $\bm{U}^{T} \bm{U} = \bm{U} \bm{U}^{T} = \bm{I}^{(m)}$ is the identity matrix of size $m$
    - $\bm{V}^{T} \bm{V} = \bm{V} \bm{V}^{T} = \bm{I}^{(n)}$ is the identity matrix of size $n$
- Matrix $S$ is formed by two vertically stacked blocks (since $m \gg n$):
    - The top block is a diagonal matrix $\Sigma \in \mathbb{R}^{n \times n}$. The diagonal elements are all non-negative (i.e. are positive or zero) and placed in decreasing order. These numbers are the *singular values* of $\bm{X}$
    - The bottom block is a matrix of zeros, of size $(m - n) \times n$

In [ ]:
from scipy.sparse.linalg import svds

In [ ]:
U, sigma, Vt = svds(sparse_matrix, k=20)

In [ ]:
U.shape, sigma.shape, Vt.shape

In [ ]:
np.savetxt(DATA_DIR / 'features.tsv', Vt.T, delimiter='\t')

In [ ]:
np.savetxt(
    DATA_DIR / 'labels.tsv',
    df['Gnrc_Name'].cat.categories.values,
    delimiter='\t',
    fmt='%s',
    encoding='utf-8',
)